In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import average_precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as porterStemmer



D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\obamaTweetsFiltered.xlsx");
data.head(5)

,date,time,Anootated tweet,Class,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,2012-10-16 00:00:00,10:28:53-05:00,"Kirkpatrick, who wore a baseball cap embroider...",0.0,NaN
2,2016-12-10 00:00:00,10:09:00-05:00,Question: If <e>Romney</e> and <e>Obama</e> ha...,2.0,NaN
3,2012-10-16 00:00:00,10:04:30-05:00,#<e>obama</e> debates that Cracker Ass Cracker...,1.0,NaN
4,2012-10-16 00:00:00,10:00:36-05:00,RT @davewiner Slate: Blame <e>Obama</e> for fo...,2.0,NaN


In [3]:
#delete classes with label = 2

data = data[data.Class != 2]

len(data)

5626

In [35]:
annotatedTweet = data['Anootated tweet']
actualClass = data['Class']
lengthObama = len(data);

annotatedTweet = annotatedTweet.drop(0)
actualClass = actualClass.drop(0)

annotatedTweetList = annotatedTweet.tolist();
actualClassList = actualClass.tolist();


# For test data

data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\testData.xlsx");
annotatedTweet = data['Anootated tweet']
annotatedTweet = annotatedTweet.drop(0)
annotatedTweetList = annotatedTweet.tolist();

In [9]:
count = 0;

finalFilteredTweet = []

def removeStopWords(tweet):
    filtered_tweet = [];
    
    stop_words = set(stopwords.words('english'));
    word_tokens = word_tokenize(tweet);
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w);
    
    eachTweet = " ".join(filtered_tweet)  
    return eachTweet
    
    
def preProcessTweets(annotatedTweet):
        
        #remove links
        processedTweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', annotatedTweet);
        #remove references with @<word>
        processedTweet = re.sub(r'(\s)@\w+', r'', processedTweet)
        processedTweet = re.sub(r'@\w+', r'', processedTweet)
        
        # remove HTML tags from tweet
        processedTweet = re.sub('<[^<]+?>', '', processedTweet)
        
        #remove exclamation marks - hashtags should be removed?
        processedTweet = re.sub(r'[<>!#@$:,%\?-]+', r' ', processedTweet)
        processedTweet = re.sub(r'[.]+', r'', processedTweet)
        
        # remove extra white spaces
        processedTweet = re.sub(r'\s+', r' ', processedTweet)
        
        #remove "" 
        processedTweet = processedTweet.replace('"', '');
        
        processedTweet = ''.join([i if ord(i) < 128 else ' ' for i in processedTweet])

        #stemming
        stemmer = porterStemmer()
        stemmedTweet = [stemmer.stem(word) for word in processedTweet.split(" ")]
        stemmedTweet = " ".join(stemmedTweet)
        processedTweet = str(stemmedTweet);
        
        processedTweet = processedTweet.replace("'", "");

        #remove numbers from data
        #join
        #processedTweet = " ".join(re.findall('[A-Z][^A-Z]*', processedTweet));

        #processedTweet = removeStopWords(processedTweet); --- Remove stop words in the end
        
        return processedTweet;

    
for every_tweet in annotatedTweetList:
    
    count = count +1     
    tweet = preProcessTweets(every_tweet).encode('ascii', 'ignore').strip();
    #print type(tweet)

    finalFilteredTweet.append(tweet);        

In [34]:
#len(actualClassList)
count

5625

In [13]:
len(finalFilteredTweet)

5625

In [10]:
finalFilteredTweet[:3]

['kirkpatrick who wore a basebal cap embroid with barack obama signatur and had the hangdog look of jason segel courier journal',
 'obama debat that cracker ass cracker tonight I will be tune in teamobama',
 'your miss the point Im afraid you do not understand the bigger pictur if you dont care if obama re elect']

In [14]:
trainData, testTrainData , y_train, y_test = train_test_split(finalFilteredTweet, actualClassList, test_size = 0.15, random_state = 40)

In [15]:
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,6),lowercase=True)

In [16]:
train_vectors = vectorizer.fit_transform(trainData);
test_vectors = vectorizer.transform(testTrainData);
total_cleanTweetVectors = vectorizer.fit_transform(finalFilteredTweet);


knn = KNeighborsClassifier()

In [37]:
testTrainData

['just in case you miss it presid obama close statement via',
 'the coup obama est la marionnett de la class dominant via',
 'I dont even care about all of thi forrealbut obama won thi one lolkilt that perfect hair mitt lol',
 'ye obama got to go',
 'when obama cant answer a question he ha the moder save him by chang the subject',
 'down goe romney down goe romney obama clear winner nbcpolit',
 'we need blue no matter what intro op campaign strategi in obama HQ philli awesom bkb',
 'MT I would ask about global warm whi obama is in bed with monstato /a while hi wife promot healthi eat debateprep',
 'RT barackobama just 21 day until elect day RT if you re readi to cheer on presid barackobama in tonight s debat teambarackobamabiden',
 'celeb hit swing state airwav for obama (blog)',
 'foxnewsradio I hear obama is work on how he can attack romney on hi tax return again',
 'fuck barack obama',
 'you are wright american vote for obama and biden who spend more time on vacat at your expens als

In [41]:
knn.fit(train_vectors, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [42]:
predicted_labels = knn.predict(test_vectors)
predicted_labelsList = predicted_labels.tolist();
#len(predicted_labelsList)
#len(y_test)

In [43]:
knn.score(test_vectors, y_test)

0.43246445497630331

In [20]:
target_classifiers = ['-1', '0', '1']

In [45]:
print(classification_report(y_test, predicted_labelsList, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.47      0.31      0.37       292
          0       0.40      0.51      0.44       284
          1       0.46      0.49      0.47       268

avg / total       0.44      0.43      0.43       844



In [46]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']

In [47]:
cvScores = cross_val_score(knn, total_cleanTweetVectors, actualClassList, cv=10,scoring='f1_macro')
print cvScores

[ 0.44033929  0.35292427  0.44797009  0.40469236  0.41944883  0.43061214
  0.50728185  0.41734905  0.43064064  0.46274363]


In [48]:
cvPredict = cross_val_predict(knn, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  0.  0. ...,  0.  1.  0.]


In [49]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.50      0.29      0.37      1968
        0.0       0.42      0.51      0.46      1978
        1.0       0.42      0.51      0.46      1679

avg / total       0.45      0.44      0.43      5625



SUPPORT VECTOR MACHINES

In [50]:
classifier_sgd = linear_model.SGDClassifier()
classifier_sgd.fit(train_vectors, y_train)




D:\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [51]:
predicted_labels = classifier_sgd.predict(test_vectors)
predicted_labelsSVMList = predicted_labels.tolist()

classifier_sgd.score(test_vectors, y_test)


0.55568720379146919

In [52]:
print(classification_report(y_test, predicted_labelsSVMList, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.55      0.59      0.57       292
          0       0.53      0.54      0.54       284
          1       0.59      0.54      0.56       268

avg / total       0.56      0.56      0.56       844



In [53]:
from sklearn.metrics import precision_score

precision_score(y_test, predicted_labelsSVMList, average='macro')  

0.55720431330187425

In [54]:
cvScores = cross_val_score(classifier_sgd, total_cleanTweetVectors, actualClassList, cv=10)
print (cvScores)

[ 0.57371226  0.48134991  0.57726465  0.57726465  0.55772647  0.59502664
  0.63410302  0.56660746  0.60249554  0.54285714]


In [56]:
cvPredict = cross_val_predict(classifier_sgd, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  1. -1. ...,  0.  0.  0.]


In [57]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.58      0.62      0.60      1968
        0.0       0.53      0.52      0.53      1978
        1.0       0.61      0.57      0.59      1679

avg / total       0.57      0.57      0.57      5625



In [68]:
bdt = AdaBoostClassifier(classifier_sgd,
                         algorithm="SAMME",
                         n_estimators=200)

In [69]:
cvPredict = cross_val_predict(bdt, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  1. -1. ...,  0.  0.  0.]


In [70]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.55      0.54      0.54      1968
        0.0       0.47      0.49      0.48      1978
        1.0       0.55      0.54      0.55      1679

avg / total       0.52      0.52      0.52      5625



MULTINOMIAL NAIVE BAYES

In [21]:
multinomialNBClassifier = MultinomialNB();
multinomialNBClassifier.fit(train_vectors, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
predicted_labels_NB = multinomialNBClassifier.predict(test_vectors)
predicted_labelsListNB = predicted_labels_NB.tolist();

print(classification_report(y_test, predicted_labelsListNB, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.55      0.63      0.59       292
          0       0.51      0.54      0.52       284
          1       0.63      0.50      0.56       268

avg / total       0.56      0.56      0.56       844



# Get Predicted list for test Data

In [11]:
submission = pd.DataFrame(columns=['id','Class'])
#submission.loc[0] = [1, 2, 3]
submission

,id,Class


In [24]:
i = 0
for num in predicted_labelsListNB:
    submission.loc[i+1] = [i+1,predicted_labelsListNB[i]]

    i=i+1

In [27]:
np.savetxt(r'C:\Users\swapnil sagar\Documents\UIC\Data mining\project 2\Swapnil_Sagar_Khusbhu_Durge_Romney.txt', submission.values, fmt='%d')
submission.head(5)

In [78]:
cvScores = cross_val_score(multinomialNBClassifier, total_cleanTweetVectors, actualClassList, cv=10)
print (cvScores)

[ 0.57726465  0.4973357   0.61811723  0.61989343  0.56838366  0.59680284
  0.660746    0.55772647  0.60606061  0.56071429]


In [28]:
cvPredict = cross_val_predict(multinomialNBClassifier, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  1. -1. ...,  0.  0.  0.]


In [29]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.58      0.68      0.63      1968
        0.0       0.55      0.51      0.53      1978
        1.0       0.64      0.58      0.61      1679

avg / total       0.59      0.59      0.59      5625



In [79]:
print ("Mean: ", cvScores.mean())

('Mean: ', 0.58630448740129038)


In [74]:
bdt = AdaBoostClassifier(multinomialNBClassifier,
                         algorithm="SAMME.R",
                         n_estimators=200)

In [115]:
cvScores = cross_val_score(bdt, total_cleanTweetVectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores.mean())

0.542021870486


In [75]:
cvPredict = cross_val_predict(bdt, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  1. -1. ...,  0.  0.  0.]


In [76]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.55      0.66      0.60      1968
        0.0       0.49      0.60      0.54      1978
        1.0       0.71      0.36      0.48      1679

avg / total       0.58      0.55      0.54      5625



Decision Tree

In [83]:
DT = DecisionTreeClassifier();
DT.fit(train_vectors, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [91]:
cvScores = cross_val_score(DT, total_cleanTweetVectors, actualClassList, cv=10)
print (cvScores.mean())

0.502567227682


In [89]:
cvPredict = cross_val_predict(DT, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  0.  0. ...,  1.  0.  0.]


In [90]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.54      0.52      0.53      1968
        0.0       0.48      0.47      0.48      1978
        1.0       0.50      0.54      0.52      1679

avg / total       0.51      0.51      0.51      5625



In [113]:
clf = RandomForestClassifier(n_estimators=10)

In [114]:
cvScores = cross_val_score(clf, total_cleanTweetVectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores.mean())

0.537824206739


In [95]:
cvPredict = cross_val_predict(clf, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  0. -1. ...,  0.  0.  0.]


In [96]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.57      0.59      0.58      1968
        0.0       0.51      0.51      0.51      1978
        1.0       0.57      0.54      0.55      1679

avg / total       0.55      0.55      0.55      5625



In [98]:
clf = GradientBoostingClassifier(n_estimators=100)

In [109]:
cvScores = cross_val_score(clf, total_cleanTweetVectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores.mean())

0.540905126596


In [99]:
cvPredict = cross_val_predict(clf, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  1.  0. ...,  0.  0.  0.]


In [100]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.57      0.54      0.55      1968
        0.0       0.47      0.62      0.54      1978
        1.0       0.64      0.46      0.53      1679

avg / total       0.56      0.54      0.54      5625



In [111]:
clf1 = LogisticRegression(random_state=1)

In [108]:
cvScores = cross_val_score(clf1, total_cleanTweetVectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores.mean())

0.583391483721


In [105]:
cvPredict = cross_val_predict(clf1, total_cleanTweetVectors, actualClassList, cv=10)
print (cvPredict)

[ 0.  1. -1. ...,  0.  0.  0.]


In [106]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.59      0.64      0.62      1968
        0.0       0.54      0.55      0.54      1978
        1.0       0.64      0.55      0.60      1679

avg / total       0.59      0.58      0.58      5625

